# idealized example regridder in the cloud

In [1]:
import xesmf as xe
import xarray as xr
import numpy as np
import dask.array as dsa

In [2]:
# from dask_gateway import GatewayCluster

# cluster = GatewayCluster()
# cluster.scale(2)
# client = Client(cluster)

In [3]:
# client

In [35]:
def da_random(nx,ny):
    nt = 3000
    y = np.linspace(-90, 90, ny)
    x = np.linspace(-180, 180, nx)
    lon, lat = np.meshgrid(x,y)
    lat = xr.DataArray(lat, dims=['y','x'])
    lon = xr.DataArray(lon, dims=['y','x'])

    data = dsa.random.random([ny, nx, nt], [ny, nx, nt/20])
    da = xr.DataArray(data, dims=['y', 'x', 'time'])
    da = da.assign_coords(lon=lon, lat=lat)
    return da

ds_source = xr.Dataset({name:da_random(180, 90) for name in ['a']})
ds_target = xr.Dataset({name:da_random(270, 120) for name in ['aa']})
ds_target

<xarray.Dataset>
Dimensions:  (y: 120, x: 270, time: 6000)
Coordinates:
    lon      (y, x) float64 -180.0 -178.7 -177.3 -176.0 ... 177.3 178.7 180.0
    lat      (y, x) float64 -90.0 -90.0 -90.0 -90.0 ... 90.0 90.0 90.0 90.0
Dimensions without coordinates: y, x, time
Data variables:
    aa       (y, x, time) float64 dask.array<chunksize=(120, 270, 300), meta=np.ndarray>

In [1]:
120//2

60

In [2]:
def da_random(nx,ny):
    nt = 1
    y = np.linspace(-90, 90, ny)
    x = np.linspace(-180, 180, nx)
    lon, lat = np.meshgrid(x,y)
    lat = xr.DataArray(lat, dims=['y','x'])
    lon = xr.DataArray(lon, dims=['y','x'])

    data = dsa.random.random([ny, nx, nt], [ny, nx, nt])
    da = xr.DataArray(data, dims=['y', 'x', 'time'])
    da = da.assign_coords(lon=lon, lat=lat)
    return da

ds_source = xr.Dataset({name:da_random(270, 120) for name in ['a']})
ds_target = xr.Dataset({name:da_random(270//4, 120//4) for name in ['aa']})
ds_target

NameError: name 'xr' is not defined

In [92]:
ds_source.a

<xarray.DataArray 'a' (y: 240, x: 540, time: 300)>
dask.array<random_sample, shape=(240, 540, 300), dtype=float64, chunksize=(240, 540, 30), chunktype=numpy.ndarray>
Coordinates:
    lon      (y, x) float64 -180.0 -179.3 -178.7 -178.0 ... 178.7 179.3 180.0
    lat      (y, x) float64 -90.0 -90.0 -90.0 -90.0 ... 90.0 90.0 90.0 90.0
Dimensions without coordinates: y, x, time

In [93]:
regridder = xe.Regridder(ds_source, ds_target, 'bilinear')

In [94]:
da_regridded = regridder(ds_source.a)

/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/gufunc.py:458: PerformanceWarning: Increasing number of chunks by factor of 10
  tmp = blockwise(


In [95]:
da_regridded.data.visualize(optimize_graph=False)

KeyboardInterrupt: 

In [ ]:
da_regridded.data.visualize(optimize_graph=True)

In [ ]:
%%time
da_regridded.isel(time=0).compute()

In [ ]:
%%time
da_regridded = regridder(ds_source.isel(time=0).a.compute())